<a href="https://colab.research.google.com/github/jdbcode/G4G19/blob/master/time-series-visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install earthengine-api
import ee
import numpy as np
import pandas as pd
import altair as alt

ee.Authenticate()
ee.Initialize()

     |████████████████████████████████| 153kB 4.9MB/s 
  Created wheel for earthengine-api: filename=earthengine_api-0.1.197-cp36-none-any.whl size=173332 sha256=0b58e71062b16cdf3a299ac67c55a532bce84b23facaeab4d64d3525b7186874
  Stored in directory: /root/.cache/pip/wheels/48/4c/32/2d2789871571a43a82315b8f1dd89447f93092050f13751810
Successfully built earthengine-api
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verificat

In [0]:
pdsi = ee.ImageCollection("IDAHO_EPSCOR/PDSI")
sn = ee.FeatureCollection("EPA/Ecoregions/2013/L3") \
  .filter(ee.Filter.eq('na_l3name', 'Sierra Nevada'))

In [0]:
def reducePDSI(img):
  eeDate = img.date()
  year = eeDate.get('year')
  month = eeDate.getRelative('month', 'year')
  doy = eeDate.getRelative('day', 'year')
  date = eeDate.format('YYYY-MM-dd')
  
  mean = img.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = sn,
    scale = 5000,
    crs = 'EPSG:5070',
    bestEffort = True,
    maxPixels = 1e14,
    tileScale = 4
  )
  
  return(ee.Feature(None, mean)
    .set({
      'DOY': doy,
      'Month': month,
      'Year': year,
      'Date': date,
      'system:time_start': img.get('system:time_start')
    })
  )

In [0]:
snPdsiCol = pdsi.map(reducePDSI) \
  .filter(ee.Filter.notNull(['pdsi']));


snPdsiDict = snPdsiCol.reduceColumns(
  reducer = ee.Reducer.toList().repeat(5),
  selectors = ['Year', 'Month', 'DOY', 'Date', 'pdsi']
)

snPdsiList = ee.List(snPdsiDict.get('list'))

In [0]:
snPdsiList = snPdsiList.getInfo()
print(snPdsiList)

[[1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983, 19

In [0]:
print('n variables:', len(snPdsiList))
print('n observations:', len(snPdsiList[0]))

n variables: 5
n observations: 1462


1. Conver the list to a Pandas dataframe.
2. Print the shape of the new dataframe. `shape` returns: (n rows, n columns)

In [0]:
snPdsiDf = pd.DataFrame(snPdsiList)
print(snPdsiDf.shape)

(5, 1462)


Transpose the dataframe and check the shape again. It should now have 4 columns and 1462 rows.

In [0]:
snPdsiDf = snPdsiDf.transpose()
print(snPdsiDf.shape)

(1462, 5)


Let's take a look at the first 10 rows of the dataframe.

In [0]:
snPdsiDf.head(5).style.hide_index()

0,1,2,3,4
1979,1,31,1979-02-01,0.0423335
1979,1,40,1979-02-10,0.0321808
1979,1,50,1979-02-20,0.0170773
1979,2,59,1979-03-01,0.0846393
1979,2,68,1979-03-10,0.0931233
1979,2,78,1979-03-20,0.0377165
1979,3,90,1979-04-01,0.114435
1979,3,99,1979-04-10,0.22713
1979,3,109,1979-04-20,0.163747
1979,4,120,1979-05-01,0.0390553


We're missing column names, let's add some.

In [0]:
snPdsiDf.columns = ['Year', 'Month', 'DOY', 'Date', 'PDSI']
snPdsiDf.head(5).style.hide_index()

Year,Month,DOY,Date,PDSI
1979,1,31,1979-02-01,0.0423335
1979,1,40,1979-02-10,0.0321808
1979,1,50,1979-02-20,0.0170773
1979,2,59,1979-03-01,0.0846393
1979,2,68,1979-03-10,0.0931233
1979,2,78,1979-03-20,0.0377165
1979,3,90,1979-04-01,0.114435
1979,3,99,1979-04-10,0.22713
1979,3,109,1979-04-20,0.163747
1979,4,120,1979-05-01,0.0390553


Now check the datatype of each column.

In [0]:
snPdsiDf.dtypes

Year     object
Month    object
DOY      object
Date     object
PDSI     object
dtype: object

Set the datatypes.

In [0]:
snPdsiDf = snPdsiDf.astype({'Year': int, 'Month': int, 'DOY': int, 'Date': str, 'PDSI': float})
snPdsiDf.dtypes
snPdsiDf['Month'] = snPdsiDf['Month'] + 1

Now let's make some charts - how about a heat map

In [0]:
alt.Chart(
  snPdsiDf,
  title="PDSI Heatmap"
).mark_rect().encode(
  x='Year:O',
  y='Month:O',
  color=alt.Color('PDSI:Q', scale=alt.Scale(scheme="redblue", domain=(-5, 5))),
  tooltip=[
    alt.Tooltip('Year:O', title='Year'),
    alt.Tooltip('mean(Month):O', title='Month'),
    alt.Tooltip('PDSI:Q', title='PDSI')
  ]
).properties(width=600)

alt.Chart(...)

In [0]:
alt.Chart(
  snPdsiDf,
  title="PDSI Barchart"
).mark_bar(size=1).encode(
  x='Date:T',
  y='PDSI:Q',
  color=alt.Color('PDSI:Q', scale=alt.Scale(scheme="redblue", domain=(-5, 5))),
  tooltip=[
    alt.Tooltip('Date:T', title='Date'),
    alt.Tooltip('PDSI:Q', title='PDSI')
  ]
).properties(width=600)

alt.Chart(...)

## Landsat NBR time series

### MODIS NDVI for phenology



In [0]:
def reduceNdvi(img):
  eeDate = img.date()
  year = eeDate.get('year')
  doy = eeDate.getRelative('day', 'year')
  
  mean = img.divide(10000).reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = sn,
    scale = 1000,
    crs = 'EPSG:5070',
    bestEffort = True,
    maxPixels = 1e14,
    tileScale = 4
  )

  return(ee.Feature(None, mean)
    .set({
      'DOY': doy,
      'Year': year
    })
  )

# get modis NDVI collection
ndvi = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')

# do the region reduction
snNdviCol = ndvi.map(reduceNdvi)

# Arrange the sample as a list of lists
snNdviDict = snNdviCol.reduceColumns(
  ee.Reducer.toList().repeat(3),
  ['Year', 'DOY', 'NDVI']
)

snNdviList = ee.List(snNdviDict.get('list'))
snNdviList = snNdviList.getInfo()

In [227]:
snNdviDf = pd.DataFrame(snNdviList)
snNdviDf = snNdviDf.transpose()
snNdviDf.columns = ['Year', 'DOY', 'NDVI']
snNdviDf = snNdviDf.astype({'Year': int, 'DOY': int, 'NDVI': float})
snNdviDf.head(5)

,Year,DOY,NDVI
0,2000,48,0.237385
1,2000,64,0.291306
2,2000,80,0.334815
3,2000,96,0.368556
4,2000,112,0.404176


Plot DOY chart

In [0]:
alt.Chart(snNdviDf).mark_line().encode(
  alt.X('DOY:O'),
  alt.Y('NDVI:Q', scale=alt.Scale(domain=(0.1, 0.7))),
  alt.Color('Year:O', scale=alt.Scale(scheme="magma")),
  tooltip=[
    alt.Tooltip('Year:O', title='Year'),  
    alt.Tooltip('DOY:O', title='DOY'),
    alt.Tooltip('NDVI:Q', title='NDVI')
  ]
).interactive().properties(width=600)

alt.Chart(...)

In [0]:
line = alt.Chart(snNdviDf).mark_line().encode(
  x='DOY:O',
  y='median(NDVI):Q',
).interactive()

band = alt.Chart(snNdviDf).mark_errorband(extent='iqr').encode(
  x='DOY:O',
  y=alt.Y('NDVI:Q', scale=alt.Scale(domain=(0.1, 0.7))),
).interactive().properties(width=600)

band + line

alt.LayerChart(...)

In [225]:
snNdviDfSub = snNdviDf[(snNdviDf['DOY'] >= 224) & (snNdviDf['DOY'] <= 272)]
snNdviDfSub = snNdviDfSub.groupby('Year').agg('min')

snPdsiDfSub = snPdsiDf[(snPdsiDf['DOY'] >= 1) & (snPdsiDf['DOY'] <= 272)]
snPdsiDfSub = snPdsiDfSub.groupby('Year').agg('mean')

test = pd.merge(snNdviDfSub, snPdsiDfSub, how='left', on='Year') \
  .drop(columns=['DOY_x', 'DOY_y', 'Month']) \
  .reset_index()

test.head(5)

,Year,NDVI,PDSI
0,2000,0.503463,-0.889241
1,2001,0.493344,-1.542462
2,2002,0.495781,-2.066630
3,2003,0.515792,-0.074927
4,2004,0.494522,-1.936395


In [224]:
test['Fit'] = np.poly1d(np.polyfit(test['PDSI'], test['NDVI'], 1))(test['PDSI'])
test.head(5)

,Year,NDVI,PDSI,Fit
0,2000,0.503463,-0.889241,0.503898
1,2001,0.493344,-1.542462,0.500240
2,2002,0.495781,-2.066630,0.497305
3,2003,0.515792,-0.074927,0.508459
4,2004,0.494522,-1.936395,0.498034


In [0]:
points = alt.Chart(test).mark_circle(size=60).encode(
  x=alt.X('PDSI:Q', scale=alt.Scale(domain=(-5, 5))),
  y=alt.Y('NDVI:Q', scale=alt.Scale(domain=(0.4, 0.6))),
  color=alt.Color('Year:O', scale=alt.Scale(scheme="viridis")),
  tooltip=['Year', 'PDSI', 'NDVI']
).interactive()

fit = alt.Chart(test).mark_line().encode(
  x=alt.X('PDSI:Q', scale=alt.Scale(domain=(-5, 5))),
  y=alt.Y('Fit:Q', scale=alt.Scale(domain=(0.4, 0.6))),
  color=alt.value('#808080')
).properties(width=600)

fit + points 

alt.LayerChart(...)

# Landsat Image Change

In [0]:
# Import the Folium library.
import folium

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
visParams = {'min':0, 'max':7000}

# Create a folium map object.
myMap = folium.Map(
  location=loc,
  #tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
  #attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',
  tiles='https://mt.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
  attr='Google',
  zoom_start=6,
  height=500)

folium.LatLngPopup().add_to(myMap)
# Display the map.
display(myMap)

In [0]:
# Start and end dates
startDay = 176
endDay = 240

Latitude = 35.9665
Longitude = -118.6407

In [0]:
point = ee.Geometry.Point([Longitude, Latitude])

# Define function to get and rename bands of interest from OLI.
def renameOLI(img):
  return(img.select(
    ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa']),
		ee.List(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])
	))

# Define function to get and rename bands of interest from ETM+.
def renameETM(img):
  return(img.select(
		ee.List(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']),
		ee.List(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])
  ))

# Define function to mask out clouds and cloud shadows.
def fmask(img):
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5
  qa = img.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)\
    .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return(img.updateMask(mask))

# Define function to calculate NBR.
def calcNBR(img):
  return(img.normalizedDifference(ee.List(['NIR', 'SWIR2'])).rename('NBR'))

# Define function to prepare OLI images.
def prepOLI(img):
  orig = img
  img = renameOLI(img)
  img = fmask(img)
  img = calcNBR(img)
  return(ee.Image(img.copyProperties(orig, orig.propertyNames())))

# Define function to prepare ETM+ images.
def prepETM(img):
  orig = img
  img = renameETM(img)
  img = fmask(img)
  #img = etm2oli(img)
  img = calcNBR(img)
  return(ee.Image(img.copyProperties(orig, orig.propertyNames())))

# get data
tmCol = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
etmCol = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
oliCol = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

# Filter collections and prepare them for merging.
oliCol = oliCol.filterBounds(point).filter(ee.Filter.calendarRange(startDay, endDay, 'day_of_year')).map(prepOLI)
etmCol = etmCol.filterBounds(point).filter(ee.Filter.calendarRange(startDay, endDay, 'day_of_year')).map(prepETM)
tmCol = tmCol.filterBounds(point).filter(ee.Filter.calendarRange(startDay, endDay, 'day_of_year')).map(prepETM)

# Merge the collections.
col = oliCol \
  .merge(etmCol) \
  .merge(tmCol)

def reduceNbr(img):
  eeDate = img.date()
  year = eeDate.get('year')
  date = eeDate.format('YYYY-MM-dd')
  
  first = img.reduceRegion(
    reducer = ee.Reducer.first(),
    geometry = point,
    scale = 30,
    crs = 'EPSG:5070'
  )

  return(ee.Feature(None, first)
    .set({
      'SATELLITE': img.get('SATELLITE'),
      'Year': year,
      'Date': date
    })
  )
  
pointNbrCol = col.map(reduceNbr).filter(ee.Filter.notNull(['NBR']))

# Arrange the sample as a list of lists
pointNbrDict = pointNbrCol.reduceColumns(
  ee.Reducer.toList().repeat(4),
  ['Year', 'Date', 'NBR', 'SATELLITE']
)

pointNbrList = ee.List(pointNbrDict.get('list'))
pointNbrList = pointNbrList.getInfo()

In [223]:
pointNbrDf = pd.DataFrame(pointNbrList)
pointNbrDf = pointNbrDf.transpose()
pointNbrDf.columns = ['Year', 'Date', 'NBR', 'Satellite']
pointNbrDf = pointNbrDf.astype({'Year': int, 'Date': str, 'NBR': float, 'Satellite': str})
pointNbrDf.head(5)

,Year,Date,NBR,Satellite
0,2013,2013-06-30,0.638326,LANDSAT_8
1,2013,2013-07-16,0.616417,LANDSAT_8
2,2013,2013-08-01,0.655117,LANDSAT_8
3,2013,2013-08-17,0.618759,LANDSAT_8
4,2014,2014-07-03,0.596919,LANDSAT_8


In [0]:
nbrSeries = alt.Chart(pointNbrDf).mark_circle(size=60).encode(
  x=alt.X('Date:T'), #, scale=alt.Scale(domain=(-5, 5))
  y=alt.Y('NBR:Q'), #, scale=alt.Scale(domain=(0.4, 0.6))
  color=alt.Color('Satellite:O', scale=alt.Scale(scheme="magma")),
  tooltip=[
    alt.Tooltip('Date:T', title='Date'),
    alt.Tooltip('NBR:Q', title='NBR'),
    alt.Tooltip('Satellite:O', title='Satellite') 
  ]
).interactive().properties(width=600)

nbrSeries

alt.Chart(...)

In [0]:
nbrSeries = alt.Chart(pointNbrDf).mark_circle(size=60).encode(
  x=alt.X('Date:T'), #, scale=alt.Scale(domain=(-5, 5))
  y=alt.Y('NBR:Q'), #, scale=alt.Scale(domain=(0.4, 0.6))
  color=alt.Color('Satellite:O', scale=alt.Scale(scheme="magma")),
  tooltip=[
    alt.Tooltip('Date:T', title='Date'),
    alt.Tooltip('NBR:Q', title='NBR'),
    alt.Tooltip('Satellite:O', title='Satellite') 
  ]
).interactive().properties(width=600)

nbrSeries

alt.Chart(...)

In [0]:
line = alt.Chart(snNdviDf).mark_line().encode(
  x='DOY:O',
  y='median(NDVI):Q',
).interactive()


line = alt.Chart(pointNbrDf).mark_line().encode(
  x=alt.X('Year:O'), #, scale=alt.Scale(domain=(-5, 5))
  y=alt.Y('median(NBR):Q') #, scale=alt.Scale(domain=(0.4, 0.6))
).interactive().properties(width=600)

band = alt.Chart(pointNbrDf).mark_errorband(extent='iqr').encode(
  x=alt.X('Year:O'),
  y=alt.Y('NBR:Q'),
).interactive().properties(width=600)

band + line

alt.LayerChart(...)

# NEX-DCP

In [0]:
dcp = ee.ImageCollection("NASA/NEX-DCP30_ENSEMBLE_STATS") \
  .select(['tasmax_median','tasmin_median', 'pr_median']) \
  .filter(ee.Filter.And(
    ee.Filter.eq('scenario', 'rcp85'),
    ee.Filter.date('2019-01-01', '2070-01-01')
  ))

def reduceDcp(img):
  eeDate = img.date()
  year = eeDate.get('year')
  date = eeDate.format('YYYY-MM-dd')
  
  altImg = img.select('tasmax_median') \
    .add(img.select('tasmin_median')) \
    .divide(ee.Image.constant(2.0)) \
    .subtract(ee.Image.constant(273.15)) \
    .addBands(img.select('pr_median')) \
    .rename(['Temp-mean', 'Precip-rate'])
  
  first = altImg.reduceRegion(
    reducer = ee.Reducer.first(),
    geometry = point,
    scale = 5000,
    crs = 'EPSG:5070'
  )

  return(ee.Feature(None, first)
    .set({
      'Year': year,
      'Date': date
    })
  )
  
pointDcpCol = dcp.map(reduceDcp).filter(ee.Filter.notNull(['Temp-mean','Precip-rate']))


# Arrange the sample as a list of lists
pointDcpDict = pointDcpCol.reduceColumns(
  ee.Reducer.toList().repeat(4),
  ['Year', 'Date', 'Temp-mean', 'Precip-rate']
)

pointDcpList = ee.List(pointDcpDict.get('list'))
pointDcpList = pointDcpList.getInfo()

In [231]:

pointDcpDf = pd.DataFrame(pointDcpList)
pointDcpDf = pointDcpDf.transpose()
pointDcpDf.columns = ['Year', 'Date', 'Temp-mean', 'Precip-rate']
pointDcpDf = pointDcpDf.astype({'Year': int, 'Date': str, 'Temp-mean': float, 'Precip-rate': float})
pointDcpDf['Precip-mm'] = pointDcpDf['Precip-rate']*86400*30
pointDcpDf = pointDcpDf.drop(columns=['Precip-rate'])
pointDcpDf['Model'] = 'NEX-DCP30'
pointDcpDf.head(5).style.hide_index()

Year,Date,Temp-mean,Precip-mm,Model
2019,2019-01-01,2.6675,142.732,NEX-DCP30
2019,2019-02-01,3.52343,140.845,NEX-DCP30
2019,2019-03-01,4.79765,99.9125,NEX-DCP30
2019,2019-04-01,7.9569,62.6253,NEX-DCP30
2019,2019-05-01,11.6865,11.2318,NEX-DCP30


## PRISM

In [0]:
prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81m') \
  .select(['ppt', 'tmean']) \
  .filter(ee.Filter.date('1979-01-01', '2019-12-31'))

def reducePrism(img):
  eeDate = img.date()
  year = eeDate.get('year')
  date = eeDate.format('YYYY-MM-dd')
    
  first = img.reduceRegion(
    reducer = ee.Reducer.first(),
    geometry = point,
    scale = 5000,
    crs = 'EPSG:5070'
  )

  return(ee.Feature(None, first)
    .set({
      'Year': year,
      'Date': date
    })
  )
  
pointPrismCol = prism.map(reducePrism).filter(ee.Filter.notNull(['ppt', 'tmean']))

pointPrismDict = pointPrismCol.reduceColumns(
  ee.Reducer.toList().repeat(4),
  ['Year', 'Date', 'tmean', 'ppt']
)

pointPrimsList = ee.List(pointPrismDict.get('list'))
pointPrimsList = pointPrimsList.getInfo()

In [232]:
pointPrimsDf = pd.DataFrame(pointPrimsList)
pointPrimsDf = pointPrimsDf.transpose()
pointPrimsDf.columns = ['Year', 'Date', 'Temp-mean', 'Precip-mm']
pointPrimsDf = pointPrimsDf.astype({'Year': int, 'Date': str, 'Precip-mm': float, 'Temp-mean': float})
pointPrimsDf['Model'] = 'PRISM'
pointPrimsDf.head(5).style.hide_index()

Year,Date,Temp-mean,Precip-mm,Model
1979,1979-01-01,-2.43,116.82,PRISM
1979,1979-02-01,-1.215,231.96,PRISM
1979,1979-03-01,1.61,197.57,PRISM
1979,1979-04-01,4.81,22.93,PRISM
1979,1979-05-01,10.32,44.14,PRISM


In [0]:
climateDf = pd.concat([pointPrimsDf, pointDcpDf])

In [235]:
line = alt.Chart(climateDf).mark_line().encode(
  x='Year:O',
  y='sum(Precip-mm):Q',
  color='Model:O',
).interactive().properties(width=600)

line

alt.Chart(...)

RCP: https://en.wikipedia.org/wiki/Representative_Concentration_Pathway

In [236]:
line = alt.Chart(climateDf).mark_line().encode(
  x='Year:O',
  y='mean(Temp-mean):Q',
  color='Model'
).interactive().properties(width=600)

line

alt.Chart(...)